In [6]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV, LassoLarsCV, BayesianRidge, ARDRegression
from sklearn.metrics import mean_squared_error, r2_score

In [7]:
###  INPUT ###
input_data = pd.read_csv('2023_smartFarm_AI_hackathon_dataset.csv') 
input_data.shape

(84840, 50)

In [8]:
df = input_data.drop(columns=['frmDist'])
df = df.sort_values(by='date')

# 데이터를 훈련 세트와 테스트 세트로 분할
'''
데이터를 훈련 세트와 테스트 세트로 나누는 데 사용됩니다. 
이는 모델의 성능을 평가하기 위해 데이터를 분리하는 일반적인 절차입니다.
'''
# 데이터셋을 data, target으로 변수분리
X = df[df.drop(columns=['outtrn_cumsum','HeatingEnergyUsage_cumsum']).columns]
Y = df[['outtrn_cumsum','HeatingEnergyUsage_cumsum']]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [9]:
class SimpleEnsemble(BaseEstimator, RegressorMixin):
    def __init__(self, models):
        self.models = [MultiOutputRegressor(model) for model in models]
        
    def fit(self, X, y):
        for model in self.models:
            model.fit(X, y)
        return self
    
    def predict(self, X):
        predictions = np.array([model.predict(X) for model in self.models])
        return np.mean(predictions, axis=0)

LR = LinearRegression()
lasso = LassoCV()
LassoLars = LassoLarsCV() 
ridge = RidgeCV() 
elasticnetcv = ElasticNetCV()
BRR = BayesianRidge()
ARDR = ARDRegression()

# 테스트 세트에 대한 예측
models = [LR, lasso, LassoLars, ridge, elasticnetcv, BRR, ARDR]
model = SimpleEnsemble(models)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

/Users/bello/anaconda3/envs/ppap/lib/python3.9/site-packages/sklearn/linear_model/_least_angle.py:675: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 46 iterations, alpha=3.014e-07, previous alpha=1.428e-07, with an active set of 37 regressors.
  warnings.warn(
/Users/bello/anaconda3/envs/ppap/lib/python3.9/site-packages/sklearn/linear_model/_least_angle.py:675: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 43 iterations, alpha=6.992e-07, previous alpha=3.690e-07, with an active set of 36 regressors.
  warnings.warn(
/Users/bello/anaconda3/envs/ppap/lib/python3.9/site-packages/sklearn/linear_model/_least_angle.py:675: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 41 iterations, alpha=1.354e-06, previous alpha=8.150e-07

In [10]:
'''
mean_squared_error와 r2_score는 회귀 모델의 예측 성능을 평가하는 메트릭입니다. 
RMSE는 예측값과 실제값 간의 제곱 오차의 평균 제곱근을 나타내며, 
R2 스코어는 모델이 데이터의 분산을 얼마나 설명하는지를 나타냅니다.
'''
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2score = r2_score(y_test, y_pred)


### OUTPUT ###
print("RMSE:", rmse)
print("R2_score:", r2score)

RMSE: 236031.04788358687
R2_score: 0.7097708594417047
